# Reto 5

Construye un sistema de recomendación usando PySpark. Para la lógica del sistema puedes emplear métodos de factorización de matrices o algún otro algoritmo que consideres pertienente. 

## Fuente de datos

- Fuente	https://www.coneval.org.mx/Paginas/principal.aspx
- GUID: indicadores-de-pobreza-pobreza-por-ingresos-rezago-social-y-gini-a-nivel-municipal1990-2000-2005-y-2010
- Publicado: 2017-10-12T22:14:14Z
- Tema: Gobiernos Locales
- Tipo del publicador: Administrador Institucional de Datos
- URL: https://datos.gob.mx/busca/dataset/indicadores-de-pobreza-pobreza-por-ingresos-rezago-social-y-gini-a-nivel-municipal1990-200-2010

## Resumen de la solución
Voy a segmentar a la población en subniveles socioeconómicos para conocer la diversidad de la pobreza. Esta segmentación ayudaría a la generación de programas sociales más especializados que facilite al pueblo la movilidad social. Por implementar, una función que asigne a qué segmento para poder representar los valores de una forma gráfica.

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

import matplotlib.pyplot as plt

In [2]:
path = "./datos/Indicadores_municipales_sabana_DA.csv"

spark = SparkSession.builder.appName("SegmentacionUsuarios").getOrCreate()

23/06/22 22:18:34 WARN Utils: Your hostname, MacBoo-Ari.local resolves to a loopback address: 127.0.0.1; using 192.168.1.85 instead (on interface en0)
23/06/22 22:18:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/22 22:18:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
usuarios_data = spark.read.csv(path, header=True, inferSchema=True)
usuarios_data.head()

23/06/22 22:18:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(ent=1, nom_ent='Aguascalientes', mun=1, clave_mun=1001, nom_mun='Aguascalientes', pobtot_ajustada=794304, pobreza=30.53110415, pobreza_e=2.264478363, pobreza_m=28.2666266, vul_car=27.98331976, vul_ing=8.419106156, npnv=33.06646943, ic_rezedu=14.97055335, ic_asalud=24.03449289, ic_segsoc=41.79988516, ic_cv=4.721187067, ic_sbv=3.214164334, ic_ali=17.6655567, carencias=58.51442354, carencias3=10.05879303, plb=38.95021004, plb_m=10.31389041, rankin_p=2351, rankin_pe=2385, N_pobreza=242510, N_pobreza_e=17987, N_pobreza_m=224523, N_vul_car=222273, N_vul_ing=66873, N_npnv=262648, N_ic_rezedu=118912, N_ic_asalud=190907, N_ic_segsoc=332018, N_ic_cv=37501, N_ic_sbv=25530, N_ic_ali=140318, N_carencias=464782, N_carencias3=79897, N_plb=309383, N_plb_m=81924, perrankin_p=22, perrankin_pe=145, cppobreza=1.905451894, cppobreza_e=3.421614885, cppobreza_m=1.784362555, cpvul_car=1.723945141, cpic_rezedu=2.111343622, cpic_asalud=2.327308178, cpic_segsoc=2.039859056, cpic_cv=2.642069578, cpic_sbv=2.78

In [4]:
# Seleccionar los indicadores de pobreza relevantes
indicadores_pobreza = usuarios_data.select("pobreza", "ic_rezedu", "ic_sbv")
indicadores_pobreza.head()

Row(pobreza=30.53110415, ic_rezedu=14.97055335, ic_sbv=3.214164334)

In [5]:
col_vect = ["pobreza", "ic_rezedu", "ic_sbv"]

aux = VectorAssembler(inputCols=col_vect, outputCol="features")
indicadores_pobreza = aux.transform(usuarios_data)

indicadores_pobreza.show()

+---+-------------------+---+---------+--------------------+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+---------+---------+-----------+-----------+---------+---------+------+-----------+-----------+-----------+-------+--------+--------+-----------+------------+------+-------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+---------+---------+---------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

In [6]:
# Modelo de clasificación
kmeans = KMeans(k=4, seed=42)

modelo_kmeans = kmeans.fit(indicadores_pobreza)
segmentos = modelo_kmeans.transform(indicadores_pobreza)

23/06/22 22:18:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [7]:
segmentos

DataFrame[ent: int, nom_ent: string, mun: int, clave_mun: int, nom_mun: string, pobtot_ajustada: int, pobreza: double, pobreza_e: double, pobreza_m: double, vul_car: double, vul_ing: double, npnv: double, ic_rezedu: double, ic_asalud: double, ic_segsoc: double, ic_cv: double, ic_sbv: double, ic_ali: double, carencias: double, carencias3: double, plb: double, plb_m: double, rankin_p: int, rankin_pe: int, N_pobreza: int, N_pobreza_e: int, N_pobreza_m: int, N_vul_car: int, N_vul_ing: int, N_npnv: int, N_ic_rezedu: int, N_ic_asalud: int, N_ic_segsoc: int, N_ic_cv: int, N_ic_sbv: int, N_ic_ali: int, N_carencias: int, N_carencias3: int, N_plb: int, N_plb_m: int, perrankin_p: int, perrankin_pe: int, cppobreza: double, cppobreza_e: double, cppobreza_m: double, cpvul_car: double, cpic_rezedu: double, cpic_asalud: double, cpic_segsoc: double, cpic_cv: double, cpic_sbv: double, cpic_ali: double, cpcarencias: double, cpcarencias3: double, cpplb: double, cpplbm: double, pobtot_00: int, pobtot_05: i

In [9]:
usuarios_segmentados = indicadores_pobreza.withColumn("prediction", segmentos["features"])
usuarios_segmentados.show()

+---+-------------------+---+---------+--------------------+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+---------+---------+-----------+-----------+---------+---------+------+-----------+-----------+-----------+-------+--------+--------+-----------+------------+------+-------+-----------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-----------+---------+---------+---------+--------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------

In [ ]:
spark.stop()